<a href="https://colab.research.google.com/github/Geuens/curso_ai/blob/main/Copy_of_embeddings_store_datastax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade astrapy
!pip install langchain langchain-openai google-cloud-secret-manager

In [ ]:
from astrapy.db import AstraDB
import langchain
from langchain_openai import OpenAIEmbeddings
import json
from google.colab import userdata
token = userdata.get('DATASTAX')
print(token)

from google.cloud import secretmanager
from google.colab import auth
auth.authenticate_user()
# Creamos un Cliente de SecretManager:
client = secretmanager.SecretManagerServiceClient()
secret_name = "openai-token" # => Cambia por el nombre de tu secreto (en google cloud secret manager)
project_id = '196177837977' # => Cambia por el id de tu proyecto Google Cloud

# Construye un F-string con los datos:
resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"

# REcupera tu secreto :
response = client.access_secret_version(request={"name": resource_name})
secret_string = response.payload.data.decode('UTF-8')

# Tu clave secreta queda bajo secret_string

# Configurar el motor de OpenAI
engine = "gpt-4"
embeddings = OpenAIEmbeddings(api_key=secret_string, model="text-embedding-3-large")

def get_embedding(text):
    query_result = embeddings.embed_query(text)
    print(query_result)
    return query_result


# Initialization
db = AstraDB(
  token=token,
  api_endpoint="https://53d8baaa-c43c-4e7e-ae0b-281fe1a9dad9-us-east-1.apps.astra.datastax.com")

print(f"Connected to Astra DB: {db.get_collections()}")

# Create a collection
collection = db.create_collection("vector_movies", dimension=3072, metric="cosine")
# Procesar cada línea y obtener los embeddings
# Leer el documento de texto

with open('combined_info_movies.csv', 'r') as file:
    lines = file.readlines()

documents = []

for index, line in enumerate(lines):
    # Extraer el texto de cada línea

    # Obtener el embedding para el texto
    vector = get_embedding(line)

    # Crear el documento
    document = {
        "_id": str(index + 1),
        "text": line,
        "$vector": vector
    }

    # Insertar el documento en la base de datos
    # Reemplaza 'collection' con tu objeto de colección de la base de datos
    # res = collection.insert_one(document)  # Usamos insert_one para insertar un solo documento
    res = collection.upsert(document)  # Aqui usamos upsert si existe se actualiza si no, se crea
    # Añadir al documento JSON
    documents.append(document)

# Guardar los documentos en un archivo JSON
with open('documentos.json', 'w') as file:
    json.dump(documents, file)
print(documents)



# Perform a similarity search
#query = [0.15, 0.1, 0.1, 0.35, 0.55]
#results = collection.vector_find(query, limit=2, fields={"text", "$vector"})

#for document in results:
#    print(document)

AstraCS:SGpbRktqFuWpREMcmHIghuLj:0087c801a5225b3c971cbaf5243d79f419bc1767437670f6104568f981bb00b4
Connected to Astra DB: {'status': {'collections': ['vector_movies']}}


[-0.003646426934636745, 0.017190930740888344, -0.036650354574940314, -0.0294372540457878, 0.026353520315945962, -0.031794360616064395, -0.03337167454821108, 0.04586611149120768, -0.04015943262239541, 0.029738538750384864, -0.01774033280887272, -0.03707569702990849, -0.005901628890989833, -0.049658751175308234, -0.00624278910149421, 0.019051804447035397, -0.022578603699797025, 0.008205568120056675, 0.06688512618945472, -0.006424445891385521, 0.040691109759783094, -0.02468759476933139, 0.02015060672035907, -0.01672128154736368, 0.039166966894136325, 0.009782880189558276, -0.03771371257500584, -0.01439075864769196, -0.017004843184009133, 0.029773983023643005, 0.021993757358554508, 0.0021898498895112856, 0.03206020104740332, -0.023695129041072304, 0.019867045083713616, 0.0022286180529175765, 0.047921935219532294, -0.004428436818905691, 0.00898536262083167, -0.0019860399631723667, 0.013743883431263888, 0.011856426054511959, -0.0014997758591043355, 0.038316281984199965, -0.019477146902003577

[-0.0225436960118693, 0.01259911312985938, -0.01013607957724769, -0.006448131394409486, -0.01941372742727164, 0.02456430847738783, -0.01377450281692596, 0.008597508525879314, -0.030745006855114408, 0.043740319530253244, 0.028341402861765584, -0.011668046619891482, -0.011918972874392121, -0.010836029996243448, -0.0028955509549245394, 0.032699586683968114, -0.03359763501295835, 0.04453271489552166, 0.013200014593857898, -0.03573710659700262, 0.035657869668179125, 0.005649130623432553, 0.004592601451875646, 0.005513762880079297, -0.0003906270054644419, -0.004090749874196993, 0.023771913112119413, 0.01458010619964354, 0.04281585516636468, -0.03420514048568145, 0.007871143934630478, -0.016864852005789247, 0.04764947918796065, 0.0005109310419370018, -0.006401907989950533, 0.0029830447572990093, 0.0053585854388588545, 0.0022533790930105053, 0.0027007534194777583, 0.019968406418133827, -0.0007304910489637487, -0.022279563602564747, -0.004209609458384042, 0.015081957777322193, -0.00335283013032

[0.008220857275797617, 0.010891768503123855, -0.010912580769373485, -0.003853743555158739, 0.0282492237010254, 0.015137476405983953, -0.014721231080991359, 0.011203952496868301, -0.004002898129947752, 0.010912580769373485, 0.06632185197303772, -0.013541867464200464, -0.017912449895880946, -0.010225775051813098, -0.0005779747922147056, 0.015484348441467054, 0.004093085082690784, 0.00914353534418714, 0.017940199584213785, -0.038211376713676526, 0.02673686196159522, -0.01756557692907524, -0.01124557702936756, 0.0038710871103667635, -0.025640749272448055, -0.0010865752471819865, -0.006996399949804554, 0.022227532019573143, 0.0006868057757680496, 0.012327816736993517, -0.005716443712807114, -0.0031582658271636886, 0.0244197611231579, -0.004713984824799374, 0.012688562685320433, 0.002336180146150057, 0.009885840438413207, 0.0048180466217088245, 0.005574226560101311, 0.0006824698869660434, -0.005255104678612352, -0.04176334054557042, -0.018675566325034035, -0.04825677506603574, -0.00586213004

[-0.017823554885670917, 0.0013283157029870683, -0.01408675376565917, -0.007252342832969452, 0.031541545922094585, -0.005789582113312879, -0.02419086521269792, 0.010700278914801643, -0.005903284016915326, 0.05202019506596412, 0.05167601552526699, -0.0510859929233162, -0.00593708729813933, -0.023355001412138456, -0.02898478628069445, -0.006385749115414525, 0.0023185985937340925, 0.015020954045662107, 0.042776528745378226, -0.08152125065770427, 0.025838007762633494, 0.018511912104420193, 0.010485166701865936, 0.019556740458135786, -0.05251187746318147, 0.012187623288657837, -0.01776209318903501, 0.004790848635090477, 0.01670497324105023, 0.004117855443516438, 0.009200641084031274, 0.0009165301819184868, 0.03041067082055972, -0.04717710203295589, -0.01618870393000453, 0.005808020436039152, 0.0372450803562657, 0.009391169177106114, -0.0034940311901560054, 0.00106403548316025, 0.020355727613242704, -0.014787403510000127, -0.03222990127819887, -0.02044177249841699, 0.0009726129090778862, -0.0

...

[-0.0100971786840453, 0.02086923282478191, -0.012433286822172788, -0.0240100008776741, -0.002087896700838338, 0.0021073643695493464, 0.0034360257159486707, 0.002048961596246966, -0.034885883516192, 0.022984709712491082, 0.03659902904953496, 0.0012564692945687822, -0.040284887169722484, -0.009597511081599841, -0.02210217912288235, 0.03151150437009029, 0.013977713957004204, 0.04160868398545817, 0.038805350866943894, -0.04625494057613073, 0.006956411096814399, -0.003851333886307715, -0.003064519741981734, 0.0332506077263705, -0.022660249964291477, -0.005129704279072551, 0.0037734636771249706, 0.017819313426879802, 0.0051816175966406175, -0.006631951348614794, -0.010590356830756443, -0.013179543497973819, 0.03971383741684473, -0.02873413093715843, -0.013004335643727966, 0.02369852004094312, 0.014704503196924871, 0.014795351851914955, 0.011570224599766997, 0.01785824829864053, -0.003961650210008807, -0.027851600347549698, -0.04508688697272827, -0.0031229222824534697, -0.010888859687341372, 

[-0.010941887652239869, 0.034062457515873945, -0.007630260459078306, 0.009008978804618054, 0.009617236774975243, -0.024451977486410866, 0.0016076600268586916, -0.01210433659034043, -0.028033941089625432, 0.026073998740696706, 0.05331044828213918, -0.01473336345172015, -0.013070791479812641, -0.013746633669098408, -0.033062209585614837, 0.0010137635167593022, -0.027641952992368726, 0.02058615792852202, 0.03738759959704375, -0.037036163148731316, 0.04217256378730315, 0.0070152438805637365, -0.031277985460842324, 0.02130255176675206, -0.01325326924344884, -0.003029463451663797, -0.009779439645461912, 0.009441517619496423, -0.0026459229161118637, 0.00032123633372715184, 0.00287401965499581, 0.021032214891037754, 0.06336698154882757, -0.021640472861394946, -0.02433032626486847, 0.001931219544828448, 0.0010923302411129682, -0.015490307076249252, -0.02979113264441364, 0.019950867388180527, -0.003666444808197893, 0.012428741399991162, -0.01424675763422796, -0.028817919146784048, -0.01148932001

[0.013132739080776775, 0.02854198447115501, -0.0066887625580240905, -0.02895811989292607, 0.011994486857298634, 0.014882955926772136, -0.016669890165830786, 0.0027201783791861024, -0.042103097483842215, 0.015348048183068442, 0.023389250396187184, -0.005532151172353011, -0.04256818880881593, 0.03483296983827839, -0.002106684757808726, 0.032678857943296516, 0.004855931344805686, 0.04007138000347993, 0.04092812600465561, -0.046558194571349465, 0.022116365247950086, -0.009846494683047739, -0.007576109956822436, 0.027587321075638483, -0.022410108117746737, 0.0024723331820411484, 0.005119075610947441, 0.02166351336443833, 0.012000606578029612, -0.011504916649401, -0.022153083572335964, -0.020867958982636905, 0.041809354614045564, -0.024331674350241753, -0.03378039277371137, 0.009558871533982067, 0.00984037496231676, 0.0071354961177887526, -0.007098778259064171, 0.03740321529238615, -0.015213416189632076, -0.03502879717637937, -0.05360801227059804, 0.0016798401740044232, -0.0234504457408518, 

[0.02076335775457333, 0.023358778638048194, -0.013339251057112122, -0.014184270748596836, -0.028489259527299762, -0.0021366944645474823, -0.012554588518035334, 0.003512870461700674, -0.04268560114758465, 0.03732575786377663, 0.034380260280929456, 0.009470264269243602, -0.04652440842792665, -0.016212319684540757, -5.564308501425735e-05, 0.01743156433803179, -0.022477542606209096, 0.022067105029138626, 0.036649740620472764, -0.016719331685696098, 0.03223149424620201, -0.027668382510513706, -0.001332416060485694, 0.022224036605631426, -0.013906622073320506, -0.008776140252746381, -0.007532753390217969, 0.021354872376802933, 0.027426948312946644, -0.02774081332857736, -7.577834058689542e-05, -0.03466997982789384, -0.0007065728887289311, -0.01734706171695753, 0.0025305342438009227, 0.02178945542253974, 0.032424639369081514, -0.0211496535882677, -0.00023464405575614845, 0.005033907396489254, -0.011866501334365977, -0.02684750549372765, -0.025447185657722603, -0.0036547132157527707, 0.0017669

KeyboardInterrupt: 